In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta
import ERFutils
import scipy.signal as signal
import cf_xarray as cfxr
from datetime import timedelta
import os
#dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
train_id = ['piControl']#,'1pctCO2','ssp126','ssp245','ssp370','ssp585','historical']
vars1 = ['rsut','rsdt','rlut','tas']
mcount = 0
for train in train_id:
    for m in ERFutils.model_set:
        for var in vars1:
            ERFutils.mf_to_sf(train, m, var)
        mcount += 1
        if mcount == 9:
            break

Checking piControl, ACCESS-ESM1-5, rsut...
Checking piControl, ACCESS-ESM1-5, rsdt...
Checking piControl, ACCESS-ESM1-5, rlut...
Checking piControl, ACCESS-ESM1-5, tas...
Checking piControl, MIROC6, rsut...
Checking piControl, MIROC6, rsdt...
Checking piControl, MIROC6, rlut...
Checking piControl, MIROC6, tas...
Checking piControl, INM-CM4-8, rsut...
Checking piControl, INM-CM4-8, rsdt...
Checking piControl, INM-CM4-8, rlut...
Checking piControl, INM-CM4-8, tas...
Checking piControl, MRI-ESM2-0, rsut...
Checking piControl, MRI-ESM2-0, rsdt...
Checking piControl, MRI-ESM2-0, rlut...
Checking piControl, MRI-ESM2-0, tas...
Checking piControl, CAMS-CSM1-0, rsut...
Checking piControl, CAMS-CSM1-0, rsdt...
Checking piControl, CAMS-CSM1-0, rlut...
Checking piControl, CAMS-CSM1-0, tas...
Checking piControl, ACCESS-CM2, rsut...
Checking piControl, ACCESS-CM2, rsdt...
Checking piControl, ACCESS-CM2, rlut...
Checking piControl, ACCESS-CM2, tas...
Checking piControl, CanESM5, rsut...
Checking piCo

In [3]:
model_set = ERFutils.model_test_set

test_climate2 = {}
for m in model_set:
    print(f'Getting climatology for {m}')
    test_climate2[m] = ERFutils.calc_climatology(m,'tas')
    break

Getting climatology for MIROC6


KeyboardInterrupt: 

In [ ]:
test_climate, test_climate_rem = {}, {}
for m in model_set:
    print(f'Getting climatology for {m}')
    test_climate_rem[m] = ERFutils.remove_climatology('1pctCO2',m,'tas')
    break

In [ ]:
#test_climate_rem[m]
A = ERFutils.A
ds_out = ERFutils.ds_out
test_climate_rem[m] = ERFutils.regrid_cmip(test_climate_rem[m],ds_out)
test_climate_rem[m] = test_climate_rem[m].weighted(A).mean(dim = ['lat','lon'])

In [ ]:
plt.plot(test_climate_rem[m].tas)

In [ ]:
# Get temperature response as difference between training and control runs
train_id = '1pctCO2'
temp_response = ERFutils.remove_climatology(train_id, m, 'tas')
temp_response = ERFutils.regrid_cmip(temp_response,ds_out)
global_temp = temp_response.weighted(A).mean(dim = ['lat','lon']).tas.values

# Have to create the patterns locally, stack data array
stacked_response = temp_response.stack(allpoints=['lat','lon'])
N_latlong = len(stacked_response['allpoints'].values)

# Convert to np arrays, xarray indexing is too slow
pattern_stacked = np.zeros((1,N_latlong))
stacked_response_np = stacked_response.tas.values

In [ ]:
from sklearn.linear_model import LinearRegression
# Calculate local patterns
for i in range(N_latlong):
    stacked_response_local = stacked_response_np[:,i]
    model = LinearRegression().fit(global_temp.reshape(-1,1), stacked_response_local.reshape(-1,1))
    pattern_stacked[0,i] = model.coef_[0][0]


In [ ]:
lat = stacked_response['lat'].values
lon = stacked_response['lon'].values
allpoints = stacked_response['allpoints'].values

me = xr.DataArray(pattern_stacked[0], coords={'lat': lat, 'lon': lon, 
                                'allpoints': allpoints},
             dims=['allpoints'])

In [ ]:
# Get pattern into the correct format
stacked_response.values = pattern_stacked
pattern = stacked_response.unstack('allpoints')
pattern.attrs = ds_train.attrs

pattern['year'] = pattern['year'] - pattern['year'][0]
        
pattern

In [ ]:
cmap = mpl.cm.RdBu_r
levels = np.linspace(-5,5,num = 2*10 + 1)
vmin = -2
vmax = 2
fig, ax= plt.subplots(figsize = [10,6], subplot_kw = {'projection':ccrs.PlateCarree()}, constrained_layout = True)

ds_dif = ds_dif.mean(dim = 'model')

# Contours of difference
conv_ds.mean(dim = 'train_id').sel(s = slice(plot_yr-yr_dif, plot_yr+yr_dif)).mean(dim = 's').plot(ax = ax, 
                                                                                                          cmap = mpl.cm.Reds, extend = 'both', 
                                                                                                          add_colorbar = True,     
                                                                                                          transform = ccrs.PlateCarree(),
                                                                                                          cbar_kwargs = {'label':r'$\Delta \overline{T}$ ($\degree$C)'})

ax.coastlines()